In [18]:
import pandas as pd
import os

In [7]:
basics_name = '/Users/JoshuaRohn/Documents/GitHub/Project3/name.basics.tsv.gz'
ratings_file = '/Users/JoshuaRohn/Desktop/SQL DATA/title.ratings.tsv.gz'
akas_file = '/Users/JoshuaRohn/Desktop/SQL DATA/title.akas.tsv.gz'

In [8]:
name_basics = pd.read_csv(basics_name, sep='\t', compression='gzip')
title_ratings = pd.read_csv(ratings_file, sep='\t', compression='gzip')
title_akas = pd.read_csv(akas_file, sep='\t', compression='gzip')

/var/folders/h7/97hfnxr93g570s_js7p5lr9h0000gn/T/ipykernel_8387/4025712473.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  title_akas = pd.read_csv(akas_file, sep='\t', compression='gzip')


In [10]:
print("title_basics columns:")
print(name_basics.columns)

print("title_ratings columns:")
print(title_ratings.columns)

print("title_akas columns:")
print(title_akas.columns)

title_basics columns:
Index(['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession',
       'knownForTitles'],
      dtype='object')
title_ratings columns:
Index(['tconst', 'averageRating', 'numVotes'], dtype='object')
title_akas columns:
Index(['titleId', 'ordering', 'title', 'region', 'language', 'types',
       'attributes', 'isOriginalTitle'],
      dtype='object')


In [14]:
filtered_movies = title_basics[
    (name_basics['primaryProfession'].str.contains('actor|actress', case=False, na=False)) & 
    (name_basics['birthYear'].notna())  ]

In [16]:
print("Filtered Movies Info:")
print(filtered_movies.info())

Filtered Movies Info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4824167 entries, 0 to 12865605
Data columns (total 6 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   nconst             object
 1   primaryName        object
 2   birthYear          object
 3   deathYear          object
 4   primaryProfession  object
 5   knownForTitles     object
dtypes: object(6)
memory usage: 257.6+ MB
None


In [19]:
output_folder = 'Data'
os.makedirs(output_folder, exist_ok=True)

In [ ]:
filtered_movies.to_csv(os.path.join(output_folder, 'filtered_movies.csv.gz'), index=False, compression='gzip')